In [1]:
import pandas as pd
import numpy as np
import keras
import tensorflow
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense


data = pd.read_csv("train.txt", sep=';')
data.columns = ["Text", "Emotions"]
data.head()

,Text,Emotions
0,i can go from feeling so hopeless to so damned...,sadness
1,im grabbing a minute to post i feel greedy wrong,anger
2,i am ever feeling nostalgic about the fireplac...,love
3,i am feeling grouchy,anger
4,ive been feeling a little burdened lately wasn...,sadness


As this is a problem of natural language processing, I’ll start by tokenizing the data:

In [2]:
texts = data["Text"].tolist()
labels = data["Emotions"].tolist()

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

Now we need to pad the sequences to the same length to feed them into a neural network. Here’s how we can pad the sequences of the texts to have the same length:

In [3]:
sequences = tokenizer.texts_to_sequences(texts)
max_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_length)

Now I’ll use the label encoder method to convert the classes from strings to a numerical representation:

In [4]:
# Encode the string labels to integers
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

We are now going to One-hot encode the labels. One hot encoding refers to the transformation of categorical labels into a binary representation where each label is represented as a vector of all zeros except a single 1. This is necessary because machine learning algorithms work with numerical data. So here is how we can One-hot encode the labels:

In [5]:
# One-hot encode the labels
one_hot_labels = keras.utils.to_categorical(labels)

## Text Emotions Classification Model
Now we will split the data into training and test sets:

In [6]:
# Split the data into training and testing sets
xtrain, xtest, ytrain, ytest = train_test_split(padded_sequences, 
                                                one_hot_labels, 
                                                test_size=0.2)

Now let’s define a neural network architecture for our classification problem and use it to train a model to classify emotions:

In [7]:
# Define the model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, 
                    output_dim=128, input_length=max_length))
model.add(Flatten())
model.add(Dense(units=128, activation="relu"))
model.add(Dense(units=len(one_hot_labels[0]), activation="softmax"))

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(xtrain, ytrain, epochs=15, batch_size=32, validation_data=(xtest, ytest))

Epoch 1/15
400/400 [==============================] - 20s 47ms/step - loss: 1.3274 - accuracy: 0.4907 - val_loss: 0.8153 - val_accuracy: 0.7031
Epoch 2/15
400/400 [==============================] - 21s 52ms/step - loss: 0.3138 - accuracy: 0.9014 - val_loss: 0.5513 - val_accuracy: 0.8156
Epoch 3/15
400/400 [==============================] - 21s 53ms/step - loss: 0.0509 - accuracy: 0.9876 - val_loss: 0.6590 - val_accuracy: 0.8028
Epoch 4/15
400/400 [==============================] - 28s 70ms/step - loss: 0.0229 - accuracy: 0.9954 - val_loss: 0.6221 - val_accuracy: 0.8194
Epoch 5/15
400/400 [==============================] - 26s 65ms/step - loss: 0.0145 - accuracy: 0.9968 - val_loss: 0.6904 - val_accuracy: 0.8150
Epoch 6/15
400/400 [==============================] - 22s 55ms/step - loss: 0.0137 - accuracy: 0.9970 - val_loss: 0.7525 - val_accuracy: 0.8069
Epoch 7/15
400/400 [==============================] - 25s 61ms/step - loss: 0.0116 - accuracy: 0.9971 - val_loss: 0.7415 - val_accuracy:

Now let’s take a sentence as an input text and see how the model performs:

In [8]:
model_architecture = model.to_json()
with open("model_architecture.json", "w") as json_file:
    json_file.write(model_architecture)

model.save_weights("model_weights.h5")

In [9]:
input_text = "She didn't come today because she lost her dog yestertay!"

# Preprocess the input text
input_sequence = tokenizer.texts_to_sequences([input_text])
padded_input_sequence = pad_sequences(input_sequence, maxlen=max_length)
prediction = model.predict(padded_input_sequence)
predicted_label = label_encoder.inverse_transform([np.argmax(prediction[0])])
print(predicted_label)

1/1 [==============================] - 0s 309ms/step
['sadness']


So this is how you can use Machine Learning for the task of text emotion classification using the Python programming language.